In [ ]:
import streamlit as st
import numpy as np
import pandas as pd
import joblib

# =========================
# Load Model + Scaler + Features
# =========================
@st.cache_resource
def load_bundle():
    bundle = joblib.load("bundle.pkl")
    return bundle["best_model"], bundle["scaler"], bundle["feature_columns"]

model, scaler, feature_columns = load_bundle()

st.set_page_config(page_title="Salary Predictor", layout="centered")
st.title("💼 Salary Prediction App")
st.write("Enter the job details below to predict expected salary.")

# =========================
# User Inputs
# =========================

# Categorical Features (from dataset)
job_titles = [
    'Data Scientist', 'Machine Learning Engineer', 'Data Analyst', 
    'Software Engineer', 'DevOps Engineer', 'Research Scientist'
]

experience_levels = ['EN', 'MI', 'SE', 'EX']
employment_types = ['FT', 'PT', 'CT', 'FL']
company_sizes = ['S', 'M', 'L']
employee_residences = ['US', 'IN', 'GB', 'CA', 'DE']

with st.form("prediction_form"):
    job_title = st.selectbox("Job Title", job_titles)
    experience = st.selectbox("Experience Level", experience_levels)
    employment_type = st.selectbox("Employment Type", employment_types)
    residence = st.selectbox("Employee Residence", employee_residences)
    company_size = st.selectbox("Company Size", company_sizes)

    remote_ratio = st.slider("Remote Ratio (%)", 0, 100, 50)
    company_location = st.text_input("Company Location (Country Code)", "US")
    year = st.number_input("Year", 2020, 2030, 2024)

    submit = st.form_submit_button("Predict Salary")

# =========================
# Preprocessing + Prediction
# =========================
if submit:

    # Prepare a single-row dataframe
    row = pd.DataFrame([[
        job_title,
        experience,
        employment_type,
        residence,
        company_location,
        company_size,
        remote_ratio,
        year
    ]], columns=feature_columns)

    # Label Encode Categorical Columns (exactly like notebook)
    for col in row.columns:
        if row[col].dtype == object:
            row[col] = row[col].astype("category").cat.codes

    # Scaling
    scaled = scaler.transform(row)

    # Prediction
    prediction = model.predict(scaled)[0]

    st.success(f"### 💰 Predicted Salary: **${prediction:,.2f}**")


# =========================
# Footer
# =========================
st.markdown(
    "<br><hr><center>Developed by Vinay Sen 🔥</center>",
    unsafe_allow_html=True
)
